In [9]:
import numpy as np
import pandas as pd
import sys
import re
import matplotlib.pyplot as plt
import uproot
import glob

sys.version

'3.7.4 (default, Aug 13 2019, 20:35:49) \n[GCC 7.3.0]'

In [12]:
#path to signal root file for skimming
signals = np.array(glob.glob('/home/aleinweber/Madgraph/MG5_aMC_v3_3_1/Generate/root_files/neg1748*.root'))
#signals = [signals[0]] #grab first file for testing purposes

processes = np.array([x.split('_')[-2] for x in signals])

In [13]:
weights_total = np.array([])

f_counter = 0
event_number = 0
for filename, process in zip(signals, processes):
    #path to csv file for saving of skimmed data
    out_file = '/home/aleinweber/Research/dimensionalReduction/non-simplified/data/sig/neg1748.63884_96.8153222_1.08875178_-71.3416523_'+process+'_gt2lep.csv'
    f_counter += 1
    with uproot.reading.open(filename) as f_in, open(out_file,'w') as f_out:
        events = f_in["Delphes;1"]
        print(filename)
        electron_Particle = events["Electron.Particle"]
        muon_Particle = events["Muon.Particle"]
        # jet_Particle = events["Jet.Particles"]
        electronE = [ [] for _ in electron_Particle.array(library='np')]
        muonE = [ [] for _ in muon_Particle.array(library='np')]
        # jetE = [ [] for _ in jet_Particle.array(library='np')]

        df = electron_Particle.arrays(library="pd")
        indices = np.array([x[0] for x in df.index])
        ref = np.array(df) 
        event_counter = 0 #count the electron-containing event we're up to
        electron_counter = 0 #count the electron we're up to
        for event_energies in events["Particle"]["Particle.E"].iterate(step_size = 1, library='np'): #iterate through each event
            if event_counter in indices: #if this event contains an electron
                event_energies = event_energies["Particle.E"][0] #convert
                mask = indices == event_counter #get indices of all electrons in this event
                refs = ref[mask] #get indices of particle array
                electronE[event_counter] = event_energies[refs-1].T[0]
                electron_counter += 1
            event_counter += 1

        df = muon_Particle.arrays(library="pd")
        indices = np.array([x[0] for x in df.index])
        ref = np.array(df) 
        event_counter = 0 #count the muon-containing event we're up to
        muon_counter = 0 #count the muon we're up to
        for event_energies in events["Particle"]["Particle.E"].iterate(step_size = 1, library='np'): #iterate through each event
            if event_counter in indices: #if this event contains an electron
                event_energies = event_energies["Particle.E"][0] #convert
                mask = indices == event_counter #get indices of all electrons in this event
                refs = ref[mask] #get indices of particle array
                muonE[event_counter] = event_energies[refs-1].T[0]
                muon_counter += 1
            event_counter += 1
       
        # Load any variables we wish to cut on
        # Load MET
        MissingET = events["MissingET"]
        MET = MissingET["MissingET.MET"].array(library='np')
        METPhi = MissingET["MissingET.Phi"].array(library='np')
                                                 
        # Load Jets
        jets = events["Jet"]
        N_jets = events["Jet_size"].array(library='np')
        jetPT = jets["Jet.PT"].array(library='np')
        jetEta = jets["Jet.Eta"].array(library='np')
        jetPhi = jets["Jet.Phi"].array(library='np')
        btag = jets["Jet.BTag"].array(library='np')
        
        # Load Electrons
        electrons = events["Electron"]
        N_electrons = events["Electron_size"].array(library='np')
        electronPT = electrons["Electron.PT"].array(library='np')
        electronEta = electrons["Electron.Eta"].array(library='np')
        electronPhi = electrons["Electron.Phi"].array(library='np')
        electronCharge = electrons["Electron.Charge"].array(library='np')
        
        # Load Muons
        muons = events["Muon"]
        N_muons = events["Muon_size"].array(library='np')
        muonPT = muons["Muon.PT"].array(library='np')
        muonEta = muons["Muon.Eta"].array(library='np')
        muonPhi = muons["Muon.Phi"].array(library='np')
        muonCharge = muons["Muon.Charge"].array(library='np')

        N_leps = N_muons + N_electrons    
        
        weights = events["Event"]["Event.Weight"].array(library='np')
        factor = np.average(weights)/sum(weights)
        
        N_events = len(weights)
        weights_total = np.append(weights_total, weights)
        print(sum(weights)*factor)
        
        #If event passes selection criterea, save it
        for i in range(N_events):
            if N_leps[i] >= 2:
                weight = factor*weights[i]
                line = str(i)+';'+process+';'+str(weight[0])+';'+str(MET[i][0]*1000)+';'+str(METPhi[i][0])+';'
                for pT, eta, phi, b in zip(jetPT[i], jetEta[i], jetPhi[i], btag[i]):
                    if b == 1:
                        line += 'b,'+'E'+','+str(pT*1000)+','+str(eta)+','+str(phi)+';'
                    elif b == 0:
                        line += 'j,'+'E'+','+str(pT*1000)+','+str(eta)+','+str(phi)+';'
                for E, pT, eta, phi, charge in zip(electronE[i], electronPT[i], electronEta[i], electronPhi[i], electronCharge[i]):
                    if charge == -1:
                        line += 'e-,'+str(E*1000)+','+str(pT*1000)+','+str(eta)+','+str(phi)+';'
                    elif charge == +1:
                        line += 'e+,'+str(E*1000)+','+str(pT*1000)+','+str(eta)+','+str(phi)+';'
                for E, pT, eta, phi, charge in zip(muonE[i], muonPT[i], muonEta[i], muonPhi[i], muonCharge[i]):
                    if charge == -1:
                        line += 'm-,'+str(E*1000)+','+str(pT*1000)+','+str(eta)+','+str(phi)+';'
                    elif charge == +1:
                        line += 'm+,'+str(E*1000)+','+str(pT*1000)+','+str(eta)+','+str(phi)+';'
                line += '\n'
                f_out.write(line)

        

        

/home/aleinweber/Madgraph/MG5_aMC_v3_3_1/Generate/root_files/neg1748.63884_96.8153222_1.08875178_-71.3416523_x1n3_jets.root
[0.18180285]
/home/aleinweber/Madgraph/MG5_aMC_v3_3_1/Generate/root_files/neg1748.63884_96.8153222_1.08875178_-71.3416523_n1n2_jets.root
[6.516298]
/home/aleinweber/Madgraph/MG5_aMC_v3_3_1/Generate/root_files/neg1748.63884_96.8153222_1.08875178_-71.3416523_x1n2_jets.root
[9.283718]
/home/aleinweber/Madgraph/MG5_aMC_v3_3_1/Generate/root_files/neg1748.63884_96.8153222_1.08875178_-71.3416523_x2n2_jets.root
[0.18814594]
/home/aleinweber/Madgraph/MG5_aMC_v3_3_1/Generate/root_files/neg1748.63884_96.8153222_1.08875178_-71.3416523_x1+x1-_jets.root
[4.852972]
